# MVP 2 Spatial Modelling and Analytics

Laure Briol
written with the help of Generative AI (ChatGPT)

In [8]:
#import statements
import os  # For file operations
import requests  # For downloading files from websites
import zipfile  # For unzipping downloaded files
import overpy  # To get info from OpenStreetMap Overpass API
import pandas as pd  # For data manipulation
import geopandas as gpd  # For geospatial data manipulation
from shapely.geometry import Point  # For creating point geometries
import numpy as np  # For numerical operations
import folium  # For creating interactive maps

# Machine learning imports
from sklearn.model_selection import train_test_split  # For splitting data
from sklearn.preprocessing import MinMaxScaler  # For scaling features
from sklearn.metrics import classification_report, accuracy_score, f1_score  # For model evaluation
from tensorflow.keras.models import Sequential  # For building neural network models
from tensorflow.keras.layers import Dense  # For defining neural network layers

In [14]:
#this MVP has a lot of code with it, so I decided to use 'functional programming'
##each step of the process is pre-coded with a 'function'. 
##then, at the bottom I can run everything in one neat little code cell.

#steps for this project:

#1. download data -- this is downloading data from the minnesota geospatial commons
## For this project we use: AADT -- annual average daily traffic data, ACS -- American community survey (demographic data), Census Tracts (population data)
#2. get store locations -- this is using the overpass api to get the locations of a store across minnesota given the store name
## To get locations of Target
#3. acs and census -- this loads in annual community survey and census tract data
## This will load the ACS data and merge it with census data. The ACS data is only in spreadsheet format, but we need to make it polygons
#4. analysis grid -- this creates a standard 'grid'/raster format so it is easy to work with
#5. data cleaning -- this takes in the data we downloaded and processes it for use with analysis, like averaging traffic within 1km of a point, or calculating population density
#6. create neural network -- this is creating our 'ai' to decide where suitable business locations could be
#7. evaluate model -- this is checking how accurate our model is at picking spots by comparing against existing store locations
#8. predict location -- this is looking at every locaiton possible, and indicating if it is a 'suitable' locaiton for a store.
#9. visualization -- this is creating a pretty map of the output and saving it to a file for later refrence


# Step 1: Download and Extract Data

#creates a function to download and extract a zip file from a given url
def download_and_extract_zip(url, extract_to='data'):
    #check if the file folder exists, if not, create it
    if not os.path.exists(extract_to):
        #create the folder
        os.makedirs(extract_to)
    
    #get the filename from the URL
    filename = url.split('/')[-1]
    #create the full path for the zip file
    zip_path = os.path.join(extract_to, filename)
    
    #check if the zip file already exists
    if not os.path.exists(zip_path):
        #print download message
        print(f"Downloading {filename}...")
        #send a request to the url with streaming so it downloads when needed
        response = requests.get(url, stream=True)
        #check if the response status is 200 (sucessful download)
        if response.status_code == 200:
            #open the zip file
            with open(zip_path, 'wb') as f:
                #download the zip file the file in chunks of data
                for chunk in response.iter_content(chunk_size=1024):
                    #if there is a chunk of data, save it
                    if chunk:
                        f.write(chunk)
            #print download completed message
            print("Download completed.")
        else:
            #error if download failed
            raise Exception(f"Failed to download file from {url}")
    else:
        #print message that file already exists, if it was already downloaded
        print(f"{filename} already exists. Skipping download.")
    
    #print extraction message
    print(f"Extracting {filename}...")
    #open the zip file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        #unzip the file to the extract_to folder
        zip_ref.extractall(extract_to)
    #print extraction completed message
    print("Extraction completed.")
    
    #return the path to where all the data is downloaded
    return extract_to


# Step 2: Get store locations

#creates a function to get store locations for a given business from openstreetmap using overpass api
def get_store_locations(business_name, area_name="Minnesota"):
    #initialize the overpass api
    api = overpy.Overpass()
    #call the openstreetmap overpass api that lets us query specific map data
    #nodes, ways, and relations are the basic data in openstreetmap
    ##nodes represent individual points defined by their latitude and longitude
    ##ways are ordered lists of nodes that define linear features like roads or boundaries
    ##relations are collections of nodes, ways, and/or other relations that define more complex structures like routes or areas
    #this defines the overpass query to get nodes, ways, and relations with the specified business name within the designated area
    query = f"""
    [out:json][timeout:60];
    area["name"="{area_name}"]->.searchArea;
    (
      node["name"="{business_name}"](area.searchArea);
      way["name"="{business_name}"](area.searchArea);
      relation["name"="{business_name}"](area.searchArea);
    );
    out center;
    """
    #try to execute the query
    try:
        result = api.query(query)
    except Exception as e:
        #print error message if the query fails
        print(f"Error fetching data for {business_name}: {e}")
        #return an empty geodataframe if there's an error
        return gpd.GeoDataFrame(columns=['lat', 'lon', 'geometry'])
    
    #initialize a list to store location data
    locations = []
    #iterate over all returned nodes, ways, and relations
    for element in result.nodes + result.ways + result.relations:
        #get the latitude attribute if available
        lat = getattr(element, 'lat', None)
        #get the longitude attribute if available
        lon = getattr(element, 'lon', None)
        #if the element has center_lat and center_lon, use them
        if hasattr(element, 'center_lat') and hasattr(element, 'center_lon'):
            lat = element.center_lat
            lon = element.center_lon
        #if both latitude and longitude are available, add them to the locations list
        if lat and lon:
            locations.append({'lat': float(lat), 'lon': float(lon)})
    
    #check if any locations were found
    if not locations:
        #print message if no locations are found
        print(f"No locations found for {business_name}.")
        #return an empty geodataframe
        return gpd.GeoDataFrame(columns=['lat', 'lon', 'geometry'])
    
    #create a dataframe from the locations list
    df = pd.DataFrame(locations)
    #convert the dataframe to a geodataframe with point geometries
    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326"
    )
    #return the geodataframe with store locations
    return gdf


# Step 3: Load ACS and Census Data

#defines a function to load acs data from an excel file
def load_acs_data(acs_excel_path):
    #print loading message
    print("Loading ACS data from Excel file...")
    #read the excel file into a dataframe
    acs_df = pd.read_excel(acs_excel_path)
    #print the number of records loaded
    print(f"Loaded ACS data with {len(acs_df)} records.")
    #return the acs dataframe
    return acs_df

#defines a function to load census tract shapefile
def load_census_tract_shapefile(shapefile_path):
    #print loading message
    print("Loading Census Tract shapefile...")
    #read the shapefile into a geodataframe
    tracts_gdf = gpd.read_file(shapefile_path)
    #print the number of records loaded
    print(f"Loaded Census Tract shapefile with {len(tracts_gdf)} records.")
    #return the census tracts geodataframe
    return tracts_gdf

#defines a function to integrate acs data with census tract geometries
def integrate_acs_with_tracts(acs_df, tracts_gdf):
    #ensure GEOID2 column in acs_df is of string type
    acs_df['GEOID2'] = acs_df['GEOID2'].astype(str)
    #ensure GEOID20 column in tracts_gdf is of string type
    tracts_gdf['GEOID20'] = tracts_gdf['GEOID20'].astype(str)
    
    #print integration message
    print("Integrating ACS data with Census Tract geometries...")
    #merge the tracts geodataframe with the acs dataframe on GEOID columns
    merged_gdf = tracts_gdf.merge(
        acs_df, left_on='GEOID20', right_on='GEOID2', how='left', suffixes=('', '_drop')
    )
    #drop any columns that have '_drop' suffix from the merge
    merged_gdf = merged_gdf[[col for col in merged_gdf.columns if not col.endswith('_drop')]]
    
    #print the number of records after merging
    print(f"Merged data has {len(merged_gdf)} records.")
    #return the merged geodataframe
    return merged_gdf


# Step 4: Create Analysis Grid

#defines a function to create a grid of points over a specified bounding box
def create_analysis_grid(minx, miny, maxx, maxy, grid_spacing=0.01):
    #create a range of x coordinates based on grid spacing
    x_coords = np.arange(minx, maxx, grid_spacing)
    #create a range of y coordinates based on grid spacing
    y_coords = np.arange(miny, maxy, grid_spacing)
    #create point geometries for each combination of x and y coordinates
    grid_points = [Point(x, y) for x in x_coords for y in y_coords]
    #create a geodataframe from the grid points with the specified CRS
    grid_gdf = gpd.GeoDataFrame(geometry=grid_points, crs="EPSG:4326")
    #print the number of grid points generated
    print(f"Generated {len(grid_gdf)} grid points.")
    #return the grid geodataframe
    return grid_gdf


# Step 5: Data Cleaning

#defines a function to calculate average traffic volume within a specified radius around each grid point
def calculate_average_traffic(grid, traffic_data, radius=1000, projection_epsg=26915):
    #print calculation message
    print("Calculating average traffic volume within 1 km radius...")
    #project traffic data to the specified EPSG for accurate distance calculations
    traffic_proj = traffic_data.to_crs(epsg=projection_epsg)
    #project grid data to the specified EPSG
    grid_proj = grid.to_crs(epsg=projection_epsg)
    
    #initialize a list to store average traffic values
    average_traffic = []
    #create a spatial index for the projected traffic data for faster queries
    traffic_sindex = traffic_proj.sindex
    
    #iterate over each point in the projected grid
    for idx, point in enumerate(grid_proj.geometry):
        #create a buffer of the specified radius around the point
        buffer = point.buffer(radius)
        #find possible matches within the buffer bounds using the spatial index
        possible_matches_index = list(traffic_sindex.intersection(buffer.bounds))
        #select the possible matching traffic records
        possible_matches = traffic_proj.iloc[possible_matches_index]
        #select the precise matches that actually intersect with the buffer
        precise_matches = possible_matches[possible_matches.intersects(buffer)]
        
        #if there are precise matches, calculate the average traffic volume
        if not precise_matches.empty:
            avg_volume = precise_matches['CURRENT_VO'].mean()
        else:
            #if no matches, set average traffic to 0
            avg_volume = 0
        
        #append the average traffic to the list
        average_traffic.append(avg_volume)
        
        #print progress every 500 grid points
        if idx % 500 == 0 and idx > 0:
            print(f"Processed {idx} grid points...")
    
    #add the average traffic data to the grid geodataframe
    grid['average_traffic'] = average_traffic
    #print completion message
    print("Average traffic calculation completed.")
    #return the updated grid geodataframe
    return grid

#defines a function to assign acs data to each grid point based on the census tract it falls into
def assign_acs_data_to_grid(grid, acs_gdf):
    #print assignment message
    print("Assigning ACS data to grid points...")
    #project acs geodataframe to match grid CRS
    acs_gdf = acs_gdf.to_crs(grid.crs)
    #perform a spatial join to assign acs data to grid points within census tracts
    grid_with_acs = gpd.sjoin(grid, acs_gdf, how='left', op='within')
    #fill any missing values with 0
    grid_with_acs.fillna(0, inplace=True)
    #print completion message
    print("Assignment completed.")
    #return the grid geodataframe with assigned ACS data
    return grid_with_acs

#defines a function to calculate additional features like population density and normalized income
def calculate_additional_features(gdf, acs_tracts_gdf):
    #calculate area in square kilometers by projecting to a suitable CRS and computing geometry area
    acs_tracts_gdf['area_sqkm'] = acs_tracts_gdf.to_crs(epsg=26915).geometry.area / 1e6
    
    #identify the correct population column name
    population_col = 'POPTOTAL' if 'POPTOTAL' in acs_tracts_gdf.columns else 'TOTAL_POP'
    #identify the correct median household income column name
    median_income_col = 'MEDIANHHI' if 'MEDIANHHI' in acs_tracts_gdf.columns else 'MED_HH_INC'
    #identify the correct drove alone column name
    drove_alone_col = 'DROVEALONE' if 'DROVEALONE' in acs_tracts_gdf.columns else 'DRIVE_ALONE'
    #identify the correct work denominator column name
    work_denom_col = 'WORKDENOM' if 'WORKDENOM' in acs_tracts_gdf.columns else 'WORK_DENOM'
    #identify the correct public transit column name
    pub_transit_col = 'PUBTRANSIT' if 'PUBTRANSIT' in acs_tracts_gdf.columns else 'PUBLIC_TRANS'
    #identify the correct walk to work column name
    walk_to_work_col = 'WALKTOWORK' if 'WALKTOWORK' in acs_tracts_gdf.columns else 'WALK_TO_WORK'
    
    #print feature calculation message
    print("Calculating additional features like population density and normalized income...")
    #select relevant columns for merging
    area_pop_df = acs_tracts_gdf[['GEOID20', 'area_sqkm', population_col, median_income_col, drove_alone_col, work_denom_col, pub_transit_col, walk_to_work_col, 'HOMEOWNPCT', 'POVERTYN']]
    #merge the selected ACS data with the grid geodataframe
    gdf = gdf.merge(area_pop_df, left_on='GEOID20', right_on='GEOID20', how='left', suffixes=('', '_drop'))
    #drop any columns with '_drop' suffix resulting from the merge
    gdf = gdf[[col for col in gdf.columns if not col.endswith('_drop')]]
    
    #calculate population density by dividing total population by area in square kilometers
    gdf['population_density'] = gdf[population_col] / gdf['area_sqkm']
    
    #normalize median household income between 0 and 1
    gdf['median_hhi_norm'] = (gdf[median_income_col] - acs_tracts_gdf[median_income_col].min()) / (acs_tracts_gdf[median_income_col].max() - acs_tracts_gdf[median_income_col].min())
    
    #calculate the percentage of people who drive alone to work
    gdf['perc_drive_alone'] = gdf[drove_alone_col] / gdf[work_denom_col]
    #calculate the percentage of people who use public transit to work
    gdf['perc_public_transit'] = gdf[pub_transit_col] / gdf[work_denom_col]
    #calculate the percentage of people who walk to work
    gdf['perc_walk_to_work'] = gdf[walk_to_work_col] / gdf[work_denom_col]
    
    #replace any infinite values with 0
    gdf.replace([np.inf, -np.inf], 0, inplace=True)
    #fill any remaining missing values with 0
    gdf.fillna(0, inplace=True)
    
    #return the geodataframe with additional features
    return gdf

#defines a function to extract features at store locations based on ACS and AADT data
def extract_features_at_locations(locations_gdf, acs_gdf, aadt_gdf):
    #ensure the CRS of acs and aadt data matches the locations geodataframe
    acs_gdf = acs_gdf.to_crs(locations_gdf.crs)
    aadt_gdf = aadt_gdf.to_crs(locations_gdf.crs)
    
    #perform a spatial join to assign ACS data to store locations
    locations_with_acs = gpd.sjoin(locations_gdf, acs_gdf, how='left', op='within')
    #fill any missing values with 0
    locations_with_acs.fillna(0, inplace=True)
    
    #calculate average traffic near each location within a 1 km radius
    locations_with_features = calculate_average_traffic(locations_with_acs, aadt_gdf, radius=1000)
    
    #return the locations geodataframe with extracted features
    return locations_with_features

#defines a function to generate negative samples from grid points not containing existing store locations
def generate_negative_samples(grid_gdf, num_samples):
    #randomly sample grid points equal to the number of negative samples needed
    negative_samples = grid_gdf.sample(n=num_samples, random_state=42)
    #return the negative samples geodataframe
    return negative_samples

#defines a function to prepare training data by combining positive and negative samples
def prepare_training_data(existing_features, negative_samples, features):
    #assign label 1 to existing store locations
    existing_features['label'] = 1
    #assign label 0 to negative samples
    negative_samples['label'] = 0
    
    #ensure both datasets have the same feature columns and the label
    existing_features = existing_features[features + ['label']]
    negative_samples = negative_samples[features + ['label']]
    
    #combine the positive and negative samples into one training dataset
    training_data = pd.concat([existing_features, negative_samples], ignore_index=True)
    
    #return the combined training data
    return training_data


# Step 6: Build and Train Neural Network

#defines a function to build and train a simple neural network model
def build_and_train_nn(X_train, y_train, input_dim, epochs=50, batch_size=16):
    #print building message
    print("Building the neural network model...")
    #define a sequential neural network model
    model = Sequential([
        #add a dense layer with 32 neurons and relu activation
        Dense(32, activation='relu', input_shape=(input_dim,)),
        #add a dense layer with 16 neurons and relu activation
        Dense(16, activation='relu'),
        #add an output dense layer with sigmoid activation for binary classification
        Dense(1, activation='sigmoid')
    ])
    
    #compile the model with adam optimizer and binary crossentropy loss
    #this is to pull everything together
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    #print training message
    print("Training the neural network model...")
    #train the model on the training data with specified epochs and batch size
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.1,
        verbose=1
    )
    
    #print completion message
    print("Training completed.")
    #return the trained model and training history
    return model, history


# Step 7: Evaluate Model

#defines a function to compute permutation importance for model features
def compute_permutation_importance(model, X_test, y_test, features, scoring='accuracy', n_repeats=5, random_state=42):
    #set the random seed for reproducibility
    np.random.seed(random_state)
    #get model predictions on the test set
    baseline_preds = model.predict(X_test).flatten()
    #convert probabilities to binary predictions
    baseline_preds_class = (baseline_preds >= 0.5).astype(int)
    
    #calculate the baseline score based on the chosen scoring method
    if scoring == 'accuracy':
        baseline_score = accuracy_score(y_test, baseline_preds_class)
    elif scoring == 'f1':
        baseline_score = f1_score(y_test, baseline_preds_class)
    else:
        #raise an error if an unsupported scoring method is chosen
        raise ValueError('Unsupported scoring method')
    
    #initialize a list to store importance scores
    importances = []
    #iterate over each feature column
    for col in range(X_test.shape[1]):
        #initialize a list to store scores for each repeat
        scores = []
        #repeat the permutation process n_repeats times
        for _ in range(n_repeats):
            #copy the test set
            X_permuted = X_test.copy()
            #shuffle the values in the current feature column
            np.random.shuffle(X_permuted[:, col])
            #get model predictions on the permuted test set
            permuted_preds = model.predict(X_permuted).flatten()
            #convert probabilities to binary predictions
            permuted_preds_class = (permuted_preds >= 0.5).astype(int)
            #calculate the score based on the chosen scoring method
            if scoring == 'accuracy':
                score = accuracy_score(y_test, permuted_preds_class)
            elif scoring == 'f1':
                score = f1_score(y_test, permuted_preds_class)
            #append the difference from the baseline score
            scores.append(baseline_score - score)
        #calculate the mean importance score for the current feature
        importances.append(np.mean(scores))
    #create a dataframe with feature names and their importance scores
    importance_df = pd.DataFrame({
        'feature': features,
        'importance_mean': importances
    }).sort_values(by='importance_mean', ascending=False)
    #return the feature importance dataframe
    return importance_df

#defines a function to evaluate the trained model on test data and print classification report
def evaluate_model(model, X_test, y_test, features):
    #print evaluation message
    print("Evaluating the model on test data...")
    #evaluate the model and get loss and accuracy
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    #print the test accuracy
    print(f'\nTest Accuracy: {accuracy:.2f}')
    
    #get model prediction probabilities on the test set
    y_pred_prob = model.predict(X_test).flatten()
    #convert probabilities to binary predictions
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    #print the classification report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    #compute feature importances using permutation importance
    importance_df = compute_permutation_importance(model, X_test, y_test, features)
    
    #print the feature importances
    print("\nFeature Importances:")
    print(importance_df)
    
    #return the feature importance dataframe
    return importance_df

# Step 8: Predict Locations

#defines a function to predict suitability for all grid points using the trained model
def predict_suitable_locations(model, scaler, grid, features):
    #print prediction message
    print("Predicting suitability for all grid points...")
    #extract feature values from the grid geodataframe
    X_grid = grid[features].values
    #scale the feature values using the provided scaler
    X_grid_scaled = scaler.transform(X_grid)
    #get model prediction probabilities for the grid points
    predictions = model.predict(X_grid_scaled).flatten()
    #add the prediction probabilities to the grid geodataframe
    grid['prediction'] = predictions
    #assign suitability based on a threshold of 0.5
    grid['suitable'] = (grid['prediction'] >= 0.5).astype(int)
    #print the number of suitable locations identified
    print(f"Identified {grid['suitable'].sum()} suitable locations.")
    #return the updated grid geodataframe with predictions
    return grid


# Step 9: Visualization

#defines a function to create an interactive folium map showing existing stores and suitable locations
def visualize_results(stores_gdf, suitable_gdf, map_filename='retail_suitability_map.html'):
    #print visualization message
    print("Creating interactive map for visualization...")
    #define the center of the map
    map_center = [44.96, -93.1]
    #create a folium map centered at the specified location
    m = folium.Map(location=map_center, zoom_start=10)
    
    #iterate over each existing store location
    for idx, row in stores_gdf.iterrows():
        #add a blue shopping cart marker for each existing store
        folium.Marker(
            [row.geometry.y, row.geometry.x],
            popup='Existing Store',
            icon=folium.Icon(color='blue', icon='shopping-cart', prefix='fa')
        ).add_to(m)
    
    #iterate over each suitable location
    for idx, row in suitable_gdf.iterrows():
        #add a green circle marker for each suitable location
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=2,
            color='green',
            fill=True,
            fill_color='green',
            fill_opacity=0.5,
            popup='Potential Site'
        ).add_to(m)
    
    #add layer control to the map
    folium.LayerControl().add_to(m)
    #save the map to an HTML file
    m.save(map_filename)
    #print completion message with the filename
    print(f"Map has been saved to '{map_filename}'.")
    #inform the user to open the map in a web browser
    print("You can open the map in a web browser to view the results.")

In [16]:
#Main function. this function is what is run when I want to make the computer do all of the predictions and everything
##I give some setup information, and it calls the functions that I created above to run the code
##I pre-defined the functions above so that this code is much shorter looking and more neat!

#defines the main function that runs everything together
def main():

    # Step 1: Download and Extract Data

    #set the url for acs data
    acs_data_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metc/society_census_acs/xlsx_society_census_acs.zip"
    #set the url for census tract shapefile
    tract_shapefile_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metc/society_census2020tiger/shp_society_census2020tiger.zip"
    #set the url for aadt data
    aadt_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/trans_aadt_traffic_count_locs/shp_trans_aadt_traffic_count_locs.zip"
    
    #set the extraction directory for acs data
    acs_extract_dir = 'data/acs'
    #download and extract acs data to the specified directory
    download_and_extract_zip(acs_data_url, extract_to=acs_extract_dir)
    #set the path to the acs excel file
    acs_excel_path = os.path.join(acs_extract_dir, 'CensusACSTract.xlsx')
    
    #set the extraction directory for census tract shapefile
    tract_extract_dir = 'data/census_tracts'
    #download and extract census tract shapefile to the specified directory
    download_and_extract_zip(tract_shapefile_url, extract_to=tract_extract_dir)
    #set the path to the census tract shapefile
    tract_shapefile_path = os.path.join(tract_extract_dir, 'Census2020TigerTract.shp')
    
    #set the extraction directory for aadt data
    aadt_extract_dir = 'data/aadt'
    #download and extract aadt data to the specified directory
    download_and_extract_zip(aadt_url, extract_to=aadt_extract_dir)
    #set the path to the aadt shapefile
    aadt_shapefile_path = os.path.join(aadt_extract_dir, 'Annual_Average_Daily_Traffic_Locations_in_Minnesota.shp')

    
    # Step 2: Get store locaitons

    #define a list of business names to fetch locations for
    businesses = ["Target"]
    #initialize a dictionary to store business locations
    business_locations = {}
    #iterate over each business in the list
    for business in businesses:
        #print fetching message for the current business
        print(f"Fetching locations for {business}...")
        #fetch store locations using the get_store_locations function
        gdf = get_store_locations(business)
        #check if the geodataframe is not empty
        if not gdf.empty:
            #store the geodataframe in the dictionary
            business_locations[business] = gdf
            #print the number of locations found
            print(f"Found {len(gdf)} locations for {business}.")
        else:
            #print message if no locations are found
            print(f"No locations found for {business}.")
    
    #check if any business locations were found
    if business_locations:
        #combine all store geodataframes into one
        all_stores_gdf = gpd.GeoDataFrame(pd.concat(business_locations.values(), ignore_index=True))
    else:
        #print message and exit if no locations are found
        print("No store locations found. Exiting script.")
        return

    
    # Step 3: Load ACS Data and Census Tract Shapefile
    
    #load acs data from the excel file
    acs_df = load_acs_data(acs_excel_path)
    #load census tract shapefile
    tracts_gdf = load_census_tract_shapefile(tract_shapefile_path)
    
    #integrate acs data with census tract geometries
    acs_tracts_gdf = integrate_acs_with_tracts(acs_df, tracts_gdf)
    

    # Step 4: Create Analysis Grid

    #print message indicating grid creation
    print("Creating analysis grid...")
    #set minimum x and y coordinates for the grid bounding box
    minx, miny = -93.65, 44.6
    #set maximum x and y coordinates for the grid bounding box
    maxx, maxy = -92.85, 45.3
    #create the analysis grid using the specified bounding box and grid spacing
    grid_gdf = create_analysis_grid(minx, miny, maxx, maxy, grid_spacing=0.01)
    
    #assign acs data to grid points based on the census tract they fall into
    grid_gdf = assign_acs_data_to_grid(grid_gdf, acs_tracts_gdf)

    
    # Step 5: Data cleaning
    
    #load aadt data from the shapefile
    aadt_gdf = gpd.read_file(aadt_shapefile_path)
    #select relevant columns and drop rows with missing values in 'CURRENT_YE' and 'CURRENT_VO'
    aadt_gdf = aadt_gdf[['geometry', 'CURRENT_YE', 'CURRENT_VO']].dropna(subset=['CURRENT_YE', 'CURRENT_VO'])
    
    #extract features at existing store locations using acs and aadt data
    existing_features = extract_features_at_locations(all_stores_gdf, acs_tracts_gdf, aadt_gdf)
    #calculate additional features like population density for existing store locations
    existing_features = calculate_additional_features(existing_features, acs_tracts_gdf)
    
    #set the number of negative samples to generate (e.g., 5 times the number of existing features)
    num_negative_samples = len(existing_features) * 5  # adjust as needed
    #generate negative samples from the grid points
    negative_samples = generate_negative_samples(grid_gdf, num_negative_samples)
    #calculate average traffic for negative samples within a 1 km radius
    negative_samples = calculate_average_traffic(negative_samples, aadt_gdf, radius=1000)
    #calculate additional features for negative samples
    negative_samples = calculate_additional_features(negative_samples, acs_tracts_gdf)
    

    # Step 6: Prepare Data for Modeling

    #print message indicating data preparation for modeling
    print("Preparing data for modeling...")
    #define the list of feature column names to be used for modeling
    features = [
        'average_traffic', 'population_density', 'median_hhi_norm',
        'perc_drive_alone', 'perc_public_transit', 'perc_walk_to_work',
        'HOMEOWNPCT', 'POVERTYN'
    ]
    #prepare the training data by combining positive and negative samples
    training_data = prepare_training_data(existing_features, negative_samples, features)
    #extract feature values from the training data
    X = training_data[features].values
    #extract labels from the training data
    y = training_data['label'].values
    
    #initialize a MinMaxScaler for feature scaling
    scaler = MinMaxScaler()
    #fit the scaler and transform the feature values
    X_scaled = scaler.fit_transform(X)
    
    #split the data into training and testing sets with stratification
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, stratify=y, random_state=42
    )
    #print the number of training and testing samples
    print(f"Training samples: {len(X_train)}, Testing samples: {len(X_test)}")
    

    # Step 7: Build and Train Neural Network

    #build and train the neural network model using the training data
    model, history = build_and_train_nn(X_train, y_train, input_dim=X_train.shape[1], epochs=50, batch_size=16)
    

    # Step 8: Evaluate Model

    #evaluate the model using the test data and get feature importances
    importance_df = evaluate_model(model, X_test, y_test, features)
    

    # Step 9: Predict Suitable Locations
    
    #calculate average traffic for all grid points
    grid_gdf = calculate_average_traffic(grid_gdf, aadt_gdf, radius=1000)
    #calculate additional features for all grid points
    grid_gdf = calculate_additional_features(grid_gdf, acs_tracts_gdf)
    #fill any missing values with 0
    grid_gdf.fillna(0, inplace=True)
    #replace any infinite values with 0
    grid_gdf.replace([np.inf, -np.inf], 0, inplace=True)
    
    #predict suitability for all grid points using the trained model
    grid_gdf = predict_suitable_locations(model, scaler, grid_gdf, features)
    #filter grid points that are predicted as suitable
    suitable_gdf = grid_gdf[grid_gdf['suitable'] == 1]
    #print the number of suitable locations identified
    print(f"Identified {len(suitable_gdf)} suitable locations.")


    # Step 10: Visualization
    
    #create an interactive map to visualize existing stores and suitable locations
    visualize_results(
        stores_gdf=all_stores_gdf,
        suitable_gdf=suitable_gdf,
        map_filename='retail_suitability_map.html'
    )
    
    #print completion message
    print("Retail Site Suitability Analysis completed successfully.")
    
#execute the script by calling the main function
if __name__ == "__main__":
    main()


xlsx_society_census_acs.zip already exists. Skipping download.
Extracting xlsx_society_census_acs.zip...
Extraction completed.
shp_society_census2020tiger.zip already exists. Skipping download.
Extracting shp_society_census2020tiger.zip...
Extraction completed.
shp_trans_aadt_traffic_count_locs.zip already exists. Skipping download.
Extracting shp_trans_aadt_traffic_count_locs.zip...
Extraction completed.
Fetching locations for Target...
Found 78 locations for Target.
Loading ACS data from Excel file...
Loaded ACS data with 1545 records.
Loading Census Tract shapefile...
Loaded Census Tract shapefile with 784 records.
Integrating ACS data with Census Tract geometries...
Merged data has 784 records.
Creating analysis grid...
Generated 5670 grid points.
Assigning ACS data to grid points...
Assignment completed.


C:\Users\Logan\AppData\Local\Temp\ipykernel_20912\3946954671.py:194: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  main()
C:\Users\Logan\AppData\Local\Temp\ipykernel_20912\3946954671.py:194: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  main()


Calculating average traffic volume within 1 km radius...
Average traffic calculation completed.
Calculating additional features like population density and normalized income...
Calculating average traffic volume within 1 km radius...
Average traffic calculation completed.
Calculating additional features like population density and normalized income...
Preparing data for modeling...
Training samples: 374, Testing samples: 94
Building the neural network model...
Training the neural network model...
Epoch 1/50
21/21 [==============================] - 1s 18ms/step - loss: 0.5886 - accuracy: 0.8482 - val_loss: 0.5354 - val_accuracy: 0.8158
Epoch 2/50
21/21 [==============================] - 0s 5ms/step - loss: 0.4712 - accuracy: 0.8363 - val_loss: 0.4575 - val_accuracy: 0.8158
Epoch 3/50
21/21 [==============================] - 0s 5ms/step - loss: 0.4136 - accuracy: 0.8363 - val_loss: 0.4276 - val_accuracy: 0.8158
Epoch 4/50
21/21 [==============================] - 0s 5ms/step - loss: 0.388